In [ ]:
#импорт библиотек
import requests
import json
import time
import os
import requests
import time
from google.colab import drive
import pandas as pd
import re
import spacy
import nltk
from IPython.display import clear_output

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
import matplotlib.ticker

In [ ]:
#загрузка словаря для лемматизации
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
#подключение google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#переход в папку с файлами
%cd "/content/drive/ProjectN5"

/content/drive/My Drive/Raw data


#1. Сбор данных вакансий

In [ ]:
# параметры GET-запроса для UX/UI дизайнера
paramsUXUI = {
     'text': 'name:[ux]', # Текст фильтра. 
     'page': 0, # Индекс страницы поиска на HH
     'per_page': 50 # Кол-во вакансий на 1 странице
}

In [ ]:
#функция для получения страницы со списком вакансий
def GetPage(page, get_params):

    #текущий номер страницы
    get_params['page'] = page

    #запрос и декодирование ответа
    request = requests.get('https://api.hh.ru/vacancies', get_params) #запрос к API
    data_page = request.content.decode() #декодирование ответа  
    request.close()

    return data_page

In [ ]:
#функция для считывания страниц и сохранения их в google drive
def ReadSavePages(page_number, get_params, path_file, path_folder):
    
    #запись вакансий со страниц
    for page in range(0, page_number):
  
        #получение информации со страницы
        page_data = GetPage(page, get_params)

        #загрузка JSON данных в словарь
        PageJSON = json.loads(page_data)
        
        #cохранение файлов в папку content\Raw Data\pagination
        page_filename = path_file.format(len(os.listdir(path_folder)))
        #page_filename = './pages/{}.json'.format(len(os.listdir('./pages')))
     
        file_current = open(page_filename, mode='w', encoding='utf8')
        file_current.write(page_data)
        file_current.close()
     
        #проверка на последнюю страницу, если вакансий меньше требуемого
        if (PageJSON['pages'] - page) <= 1:
           break
     
        #задержка, чтобы не нагружать сервисы hh
        time.sleep(0.25)

    print('100% страниц поиска собраны')


In [ ]:
#метод для чтения и сохранения вакансий
def ReadSaveVacancy(path_file_read, path_folder_read, path_file_write):
    
    #получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
    #i = 0
    for filename in os.listdir(path_folder_read):

        #чтение содержимого файла
        current_file = open(path_file_read.format(filename), encoding='utf8', errors='ignore')
        #current_file = open('./pages/{}'.format(filename), encoding='utf8', errors='ignore')
        jsonText = current_file.read()
        current_file.close()
     
        #преобразование полученного текста
        DataJSON = json.loads(jsonText)
     
        #анализ вакансий на странице
        for vacancy in DataJSON['items']:
         
            #обращение к API и получение информации по вакансии
            req = requests.get(vacancy['url'])
            data_vacancy = req.content.decode()
            req.close()
         
            #cоздание файла в формате json с идентификатором вакансии в качестве названия
            fileName = path_file_write.format(vacancy['id'])
            f = open(fileName, mode='w', encoding='utf8')

            #запись данных в файл
            f.write(data_vacancy)
            f.close()
            #i += 1
         
        time.sleep(0.25)
         
    print('Вакансии собраны')

In [ ]:
def ListsFiling(list_to_fill, text, length):

    list_to_fill = list_to_fill.clear()
    list_to_fill = [text] * length
  
    return list_to_fill

In [ ]:
#функция для извлечения информации о вакансии
def VacancyData():

    files_number = len(os.listdir('./vacancies/'))
    i = 0

    #вспомогательные списки
    id_current = []
    name_current = []
    experience_current = []
    position_current = []

    #глобальные списки для навыков соискателя
    ids = [] #идентификатор вакансии
    names = [] #наименование вакансии
    descriptions = [] #описание вакансии
    experiences = [] #опыт требуемый у соискателя
    skills_name = [] #название ключевого навыка
    positions = [] #позиция (junior, middle, senior и т.д.)

    # Проходимся по всем файлам в папке vacancies
    for files in os.listdir('./vacancies/'):
    #for i in range(0,10):
     
        #чтение данных
        f_data = open('./vacancies/{}'.format(files), encoding='utf8')
        jsonText = f_data.read()
        f_data.close()
     
        #перевод данных из JSON
        DataJSON = json.loads(jsonText)
     
        # Заполняем списки для таблиц
        id = DataJSON['id']
        name = DataJSON['name']
        descriptions.append(DataJSON['description'])
        experience = DataJSON['experience']['id']

        for skill in DataJSON['key_skills']:
            skills_name.append(skill['name']) 
        
        n_length = len(DataJSON['key_skills'])

        #заполнение текущего листа для вакансии по числу навыков
        id_current = ListsFiling(id_current, id, n_length)
        name_current = ListsFiling(name_current, name, n_length)
        experience_current = ListsFiling(experience_current, experience, n_length)

        position = NameAnalysis(name, experience)
        position_current = ListsFiling(position_current, position, n_length)

        #запись данных в глобальные списки
        ids.extend(id_current)
        names.extend(name_current)
        experiences.extend(experience_current)  
        positions.extend(position_current)     
        
        #отображение прогресса
        i += 1
        clear_output(wait=True)
        display('Готово {} из {}'.format(i, files_number))

        print('ids length', len(ids))
        print('names length', len(names))
        print('exp length', len(experiences))
        print('skills length', len(skills_name))

        #создание датафрейма со навыками и данными опыта
        df_skill = pd.DataFrame({'id': ids, 'name': names, 
                                 'position' : positions, 'experience' : experiences, 'skills' : skills_name})
        df_skill.to_csv('./database/skills_UXUI.csv')

In [ ]:
#функция для лемматизации текста
def lemmatize(text):
    
    #лемматизация
    text_ = nlp(text)
    
    return " ".join([token.lemma_ for token in text_])

In [ ]:
#функция для перевода описания опыта в годы
def ExperienceToYears(experience_data):

    #noExperience значение - стажер
    #between1and3  - middle
    #between3and6 - middle+

    if experience_data == 'noExperience':
        return 'стажер'
    elif experience_data == 'between1And3':
        return 'Middle'
    else:
        return 'Middle+'

In [ ]:
#очистка текста от лишних символов
def clear_text(text):
    
    text_cleared = re.sub(r"[^a-zA-Zа-яА-ЯёЁ\' ]", ' ', text) #очистка текста
    
    return " ".join(text_cleared.split())  

In [ ]:
jun = ['Junior', 'junior', 'младший']
middle = ['Middle', 'middle']
senior = ['Senior', 'senior']
lead = ['Lead', 'lead']
manager = ['Менеджер', 'менеджер']
reviewer = ['reviewer', 'Reviewer']

In [ ]:
#функция для извлечения уровня вакансии из описания
def NameAnalysis(vacancy_name, experience):

    # #загрузка стоп-слов
    # nltk.download('stopwords')
    # stopwords = set(stopwords.words('english'))

    #создание корпуса
    text = lemmatize(clear_text(vacancy_name)).split()
    #print(text)
    
    if not set(stop).isdisjoint(set(text)):  
      return 'Stop'
    elif not set(jun).isdisjoint(set(text)):
      return 'Junior'
    elif not set(middle).isdisjoint(set(text)):
      return 'Middle'
    elif not set(senior).isdisjoint(set(text)):
      return 'Senior'
    elif not set(lead).isdisjoint(set(text)):
      return 'Lead'
    elif not set(manager).isdisjoint(set(text)):
      return 'Manager'
    elif not set(reviewer).isdisjoint(set(text)):
      return 'Reviewer'
    else:
      return ExperienceToYears(experience)

In [ ]:
df = pd.read_csv('./database/skills_UXUI.csv')

In [ ]:
def DataDuplicates(data):
  
  #приведение описания навыков к строчным буквам, в случае возможно разного их описания
  data['skills'] = data['skills'].str.lower()

  data.loc[(data.loc[:,'skills'] == 'web design') | 
           (data.loc[:,'skills'] == 'web-дизайн'), 'skills'] = 'веб-дизайн'
  
  return data  

In [ ]:
#создание датасетов под различные позиции
df_intern = df.query('position == "стажер"')
df_junior = df.query('position == "Junior"')
df_middle = df.query('position =="Middle"')
df_middle_plus = df.query('position == "Middle+"')
df_senior = df.query('position == "Senior"')
df_lead = df.query('position == "Lead"')
df_list = [df_intern, df_junior, df_middle, df_middle_plus, df_senior, df_lead]
df_list_pivot = []

In [ ]:
#функция для построения сводных таблиц для датасетов
def VacancyPivot(frames_list):

  df_list_pivot = []
  
  #расчет частоты навыков в цикле
  for i in range(len(frames_list)):
    df_curr = pd.DataFrame()
    df_curr = frames_list[i]

    df_curr_pivot = df_curr.pivot_table(values = 'id', index = 'skills', aggfunc = ['count']).reset_index()
    df_curr_pivot.columns = ['skills','sum']
    
    df_curr_pivot = df_curr_pivot.reset_index(drop = True)
    
    df_curr_pivot = df_curr_pivot.sort_values(by = 'sum', ascending = False)
    df_curr_pivot_S = df_curr_pivot['sum'].sum()
    df_curr_pivot['frequency'] = df_curr_pivot['sum'] / df_curr_pivot_S
    df_curr_pivot['percent'] = pd.Series(['{:.2%}'.format(x) for x in df_curr_pivot['frequency']], index = df_curr_pivot.index)

    df_list_pivot.append(df_curr_pivot)
    del df_curr
    del df_curr_pivot

  return df_list_pivot

In [ ]:
 #чтение страниц с вакансиями
 ReadSavePages(20, paramsUXUI)

100% страниц поиска собраны


In [ ]:
ReadSaveVacancy()

In [ ]:
VacancyData()

'Готово 893 из 893'

ids length 4734
names length 4734
exp length 4734
skills length 4734


In [ ]:
df = DataDuplicates(df)
df

,Unnamed: 0,id,name,position,experience,skills
0,0,44533913,Стажер UI/UX Designer,стажер,noExperience,adobe illustrator
1,1,44533913,Стажер UI/UX Designer,стажер,noExperience,adobe photoshop
2,2,44533913,Стажер UI/UX Designer,стажер,noExperience,adobe after effect
3,3,44533913,Стажер UI/UX Designer,стажер,noExperience,ui
4,4,44533913,Стажер UI/UX Designer,стажер,noExperience,ux
...,...,...,...,...,...,...
4729,4729,44588540,UX-аналитик (количественные исследования),Middle,between1And3,surveymonkey
4730,4730,44588540,UX-аналитик (количественные исследования),Middle,between1And3,google analytics
4731,4731,44588540,UX-аналитик (количественные исследования),Middle,between1And3,яндекс.метрика
4732,4732,44588540,UX-аналитик (количественные исследования),Middle,between1And3,аналитический склад ума


In [ ]:
data_list = VacancyPivot(df_list)
#data_list[4]

In [ ]:
csv_names  = ['стажер', 'junior','middle', 'middle+', 'senior', 'lead']
for i in range(len(data_list)):
  filename = './database/{}.csv'.format(csv_names[i])
  data_list[i].to_csv(filename)

In [ ]:
#функция для построения графиков навыков
def GraphBuild(data, position_name):

    columns_order = []
    
    #построение графиков
    fig, axes = plt.subplots(1, 1, figsize=(16,8))
    plt.suptitle(position_name)
    
    #построение столбчатой диаграммы
    freq_threshold = data[data['frequency'] > 0.005]
    axes.bar(freq_threshold['skills'], (freq_threshold['frequency']  * 100), color="blue", align = "center", )
    
    axes.set_xticklabels(data['skills'], rotation = 85)
    #axes.set_xlim([-1, features.shape[1]])


In [ ]:
#функция для построения таблицы скиллов
def SkillSetTable(data, data_list, names_list, fill_value_p, fill_value_n):

  full_skill_list = data['skills'].to_list()
  full_freq = data['frequency'].to_list()
  full_percent = data['percent'].to_list()
  
  #создвние датафрейма с полным набором наывков
  skills_df = pd.DataFrame({'skills': full_skill_list})

  #цикл по всем датафреймам с позициями
  for i in range(len(data_list)):
    position_skill = []
    freq = []
    percent =[]

    skill_list_current = data_list[i]['skills'].to_list()


    for j in range(len(full_skill_list)):
        if full_skill_list[j] in skill_list_current:
            position_skill.append(fill_value_p)
            freq.append(full_freq[j])
            percent.append(full_percent[j])
        else:
            position_skill.append(fill_value_n)
            freq.append(full_freq[j])
            percent.append(full_percent[j])
    
    #positions_list.append(position_skill)
    skills_df[names_list[i]] = position_skill
    

    skill_list_current = skill_list_current.clear()
    position_skill = position_skill.clear()
    

  skills_df['frequency'] = freq
  skills_df['percent'] = percent

  freq = freq.clear()
  percent = percent.clear()

  return skills_df


In [ ]:
GraphBuild(data_list[4], 'Senior UX/UI')

In [ ]:
GraphBuild(data_list[0], 'Стажер UX/UI')

In [ ]:
GraphBuild(data_list[3], 'Middle+ UX/UI')

In [ ]:
GraphBuild(data_list[2], 'Middle UX/UI')

In [ ]:
GraphBuild(data_list[5], 'Lead UX/UI')

In [ ]:
GraphBuild(data_list[1], 'Junior UX/UI')

#2. Сбор данных для их анализа в ML модели
Данные для позиции Data Analyst и Data Scientist


In [ ]:
dfs= SkillSetTable(dfs_pivot[0], df_list, csv_names, "+", "-")
dfs.to_csv('/content/drive/My Drive/Raw data/skillset_full.csv')
#dfs.head(25)

In [ ]:
#переход в папку с файлами
%cd "/content/drive/My Drive/ML model"

In [ ]:
# параметры GET-запроса для Data Analyst
paramsDA = {
     'text': 'name:"Data Analyst" or "дата аналитик" or "data analyst" or "Data аналитик"', # Текст фильтра. 
     #'area': 'name:[Москва, Санкт-Петербург]', # Поиск ощуществляется по вакансиям города Москва
     'page': 0, # Индекс страницы поиска на HH
     'per_page': 50 # Кол-во вакансий на 1 странице
}

In [ ]:
#список путей
pages_file_DA = './DA data/pages/{}.json'
pages_folder_DA = './DA data/pages/'
page_file_DA_read = './DA data/pages/{}'
vacancies_file_DA = './DA data/vacancies/{}.json'

In [ ]:
# параметры GET-запроса для Data Scientist
paramsDS = {
     'text': 'name:"Data Scientist" or "data scientist" or "Data scientist"', # Текст фильтра. 
     #'area': 'name:[Москва, Санкт-Петербург]', # Поиск ощуществляется по вакансиям города Москва
     'page': 0, # Индекс страницы поиска на HH
     'per_page': 50 # Кол-во вакансий на 1 странице
}

In [ ]:
#список путей
pages_file_DS = './DS data/pages/{}.json'
pages_folder_DS = './DS data/pages/'
page_file_DS_read = './DS data/pages/{}'
vacancies_file_DS = './DS data/vacancies/{}.json'

In [ ]:
#чтение страниц с вакансиями
ReadSavePages(20, paramsDS, pages_file_DS, pages_folder_DS)

100% страниц поиска собраны


In [ ]:
ReadSaveVacancy(page_file_DS_read, pages_folder_DS, vacancies_file_DS)

Вакансии собраны


In [ ]:
#функция для извлечения информации о вакансии
def VacancyFeatureExtraction():

    files_number = len(os.listdir('./vacancies/'))
    i = 0


    df = pd.DataFrame(columns=['id', 'name', 'skills'])

    #вспомогательные списки
    id_current = []
    name_current = []
    # experience_current = []
    # position_current = []

    #глобальные списки для навыков соискателя
    
    # descriptions = [] #описание вакансии
    # experiences = [] #опыт требуемый у соискателя
    # positions = [] #позиция (junior, middle, senior и т.д.)

    # Проходимся по всем файлам в папке vacancies
    for files in os.listdir('./vacancies/'):
    #for i in range(0,10):

        ids = [] #идентификатор вакансии
        names = [] #наименование вакансии
        skills_name = [] #название ключевого навыка

        #чтение данных
        f_data = open('./vacancies/{}'.format(files), encoding='utf8')
        jsonText = f_data.read()
        f_data.close()
     
        #перевод данных из JSON
        DataJSON = json.loads(jsonText)
     
        # Заполняем списки для таблиц
        id = DataJSON['id']
        name = DataJSON['name']
        # descriptions.append(DataJSON['description'])
        # experience = DataJSON['experience']['id']

        for skill in DataJSON['key_skills']:
            ids.append(id)
            names.append(name)
            skills_name.append(skill['name']) 
        
        n_length = len(DataJSON['key_skills'])

        # заполнение текущего листа для вакансии по числу навыков
        # id_current = ListsFiling(id_current, id, n_length)
        # name_current = ListsFiling(name_current, name, n_length)
        # experience_current = ListsFiling(experience_current, experience, n_length)

        # position = NameAnalysis(name, experience)
        # position_current = ListsFiling(position_current, position, n_length)

        #запись данных в глобальные списки
        ids.extend(id_current)
        names.extend(name_current)
        # experiences.extend(experience_current)  
        # positions.extend(position_current)     
        
        #отображение прогресса
        i += 1
        clear_output(wait=True)
        display('Готово {} из {}'.format(i, files_number))

        print('ids length', len(ids))
        print('names length', len(names))
        # print('exp length', len(experiences))
        print('skills length', len(skills_name))

        #создание датафрейма со навыками и данными опыта
        df_skill = pd.DataFrame({'id': ids, 'name': names, 'skills' : skills_name})
        df=df.append(df_skill)
        # df_skill.to_csv('./database/skills_UXUI.csv')
    return df
    

In [ ]:
df.to_csv('/content/drive/MyDrive/ML model/skills_for_prediction_model.csv')